In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torchvision
import torch

/opt/homebrew/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9DBE5D5C-AC87-30CA-96DA-F5BC116EDA2B> /opt/homebrew/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <44DEDA27-4DE9-3D4A-8EDE-5AA72081319F> /opt/homebrew/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from glob import glob
f_names = {}
for filename in glob('../data/**/*.JPG', recursive=True):  # data содержит папки D850-Vol1 и тд
    f_names[filename.split('/')[-1]] = {'path': filename} # \\ -> /


markup_updated = pd.read_json('../artifacts/markup_updated.json').T.to_dict()

areas = []
for f in markup_updated:
    for m in markup_updated[f]['coco']:
        areas.append(m[-1] * m[-2])

In [3]:
import sys
sys.path.append('../')

import joblib

sets_info = joblib.load('../artifacts/sets_info_v3.joblib')
save_path = 'preds_all'
results = joblib.load('../artifacts/res_8s_slicing_0.45.joblib')

fname2index = {j:i for i,j in dict(enumerate(f_names.keys())).items()}
index2fname = {j:i for i,j in fname2index.items()}

In [4]:
print(len(sets_info['train']['names']))
print(len(sets_info['val']['names']))

143
62


In [5]:
sys.path.append('../utils')
from metrics import calc_tp_fp_fn, get_true_boxes_format, get_predicted_boxes_format, print_main_metrics

confidence_threshold = 0.45
min_area = np.quantile(areas, 0.02)
max_area=np.quantile(areas, 0.98)
score_threshold = 0.8265
iou_threshold=0.2

detections = get_predicted_boxes_format({i: results[i] for i in results}, fname2index, min_area=min_area, max_area=max_area)
detections = [i for i in detections if i[2] > score_threshold]

ground_truths = get_true_boxes_format({i: markup_updated[i] for i in markup_updated}, fname2index)

tp_list, fp_list, fn = calc_tp_fp_fn(ground_truths, detections, iou_threshold)


print_main_metrics(*calc_tp_fp_fn(ground_truths, [i for i in detections if i[2] > score_threshold], iou_threshold))

TP 297
recall 0.871
precision 0.616
f_score 0.722


In [6]:
from random import random


def classify_crop(crop_img, model=None):
    return random()


detections_after_classifier = []

for detection in tqdm(detections):

    img = cv2.imread(f_names[index2fname[detection[0]]]['path'])
    x_min, y_min, x_max, y_max = [int(i) for i in detection[3:]]

    crop_img = img[y_min:y_max, x_min:x_max]
    
    
    # Вызываем классификатор
    if classify_crop(crop_img=crop_img) > 0.5:
        detections_after_classifier.append(detection)

100%|█████████████████████████████████████████| 482/482 [02:32<00:00,  3.17it/s]


In [7]:
print_main_metrics(*calc_tp_fp_fn(ground_truths, detections_after_classifier, iou_threshold))

TP 144
recall 0.422
precision 0.621
f_score 0.503


In [8]:
from torchvision import transforms
from PIL import Image

In [9]:
data_transform_resize = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()
])

class ResNetPad():
    def __call__(self, image, shape=(224, 224)):
        y_, x_ = shape
        y, x = image.size[1], image.size[0]
        y_pad = (y_-y)
        x_pad = (x_-x)
        return np.pad(image, ((y_pad//2, y_pad//2 + y_pad%2),
                         (x_pad//2, x_pad//2 + x_pad%2),
                         (0, 0)),
                      mode = 'constant')

data_transform_padding = transforms.Compose([
    ResNetPad(),
    transforms.ToTensor()
])

In [21]:
resnet50_padding = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights)
resnet50_resize = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights)
resnet50_knn = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights)
resnet50_resize_knn = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights)

resnet50_padding_dict = torch.load('./resnet50_padding_split.pth', map_location=torch.device('cpu'))
resnet50_resize_dict = torch.load('./resnet50_resize_split.pth', map_location=torch.device('cpu'))

resnet50_padding.load_state_dict(resnet50_padding_dict)
resnet50_resize.load_state_dict(resnet50_resize_dict)
resnet50_resize_knn.load_state_dict(resnet50_resize_dict)

resnet50_knn.fc = torch.nn.Identity()
resnet50_resize_knn.fc = torch.nn.Identity()

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
import pickle

knn = pickle.load(open('./knnpickle_file', 'rb'))

In [12]:
def classify_crop(crop_img, model=None, transform=None):
    model.eval()
    
    im = Image.fromarray(np.uint8(crop_img[:, :, [2, 1, 0]]))
    transformed_image = transform(im)
    transformed_image = transformed_image.unsqueeze(0)
    with torch.no_grad():
        outputs = model(transformed_image)
    prediction = torch.argmax(outputs, dim=1)
    return prediction.item()

## Resize

In [12]:
detections_after_classifier = []
predictions = []

for detection in tqdm(detections):

    img = cv2.imread(f_names[index2fname[detection[0]]]['path'])
    x_min, y_min, x_max, y_max = [int(i) for i in detection[3:]]

    crop_img = img[y_min:y_max, x_min:x_max]
    
    prediction = classify_crop(crop_img=crop_img, model=resnet50_resize, 
                    transform=data_transform_resize)
    predictions.append(prediction)
    
    # Вызываем классификатор
    if classify_crop(crop_img=crop_img, model=resnet50_resize, 
                    transform=data_transform_resize) > 0.5:
        detections_after_classifier.append(detection)

100%|█████████████████████████████████████████| 482/482 [03:08<00:00,  2.55it/s]


In [13]:
print_main_metrics(*calc_tp_fp_fn(ground_truths, detections_after_classifier, iou_threshold))

TP 297
recall 0.871
precision 0.675
f_score 0.761


## Padding

In [14]:
detections_after_classifier = []

for detection in tqdm(detections):

    img = cv2.imread(f_names[index2fname[detection[0]]]['path'])
    x_min, y_min, x_max, y_max = [int(i) for i in detection[3:]]

    crop_img = img[y_min:y_max, x_min:x_max]
    
    
    # Вызываем классификатор
    if classify_crop(crop_img=crop_img, model=resnet50_padding, #what the hell?
                     transform=data_transform_padding) > 0.5:
        detections_after_classifier.append(detection)

100%|█████████████████████████████████████████| 482/482 [02:52<00:00,  2.79it/s]


In [15]:
print_main_metrics(*calc_tp_fp_fn(ground_truths, detections_after_classifier, iou_threshold))

TP 297
recall 0.871
precision 0.632
f_score 0.732


## KNN

In [32]:
def classify_crop(crop_img, model=None, knn=None, transform=None):
    model.eval()
    
    im = Image.fromarray(np.uint8(crop_img[:, :, [2, 1, 0]]))
    transformed_image = transform(im)
    transformed_image = transformed_image.unsqueeze(0)
    with torch.no_grad():
        outputs = model(transformed_image)
    prediction = knn.predict(outputs.cpu().numpy())
    return prediction.item()

In [33]:
detections_after_classifier = []
predictions = []

for detection in tqdm(detections):

    img = cv2.imread(f_names[index2fname[detection[0]]]['path'])
    x_min, y_min, x_max, y_max = [int(i) for i in detection[3:]]

    crop_img = img[y_min:y_max, x_min:x_max]
    
    prediction = classify_crop(crop_img=crop_img, model=resnet50_knn, knn=knn,
                    transform=data_transform_resize)
    predictions.append(prediction)
    
    # Вызываем классификатор
    if classify_crop(crop_img=crop_img, model=resnet50_knn, knn=knn,
                    transform=data_transform_resize) > 0.5:
        detections_after_classifier.append(detection)

100%|█████████████████████████████████████████| 482/482 [04:31<00:00,  1.78it/s]


In [34]:
print_main_metrics(*calc_tp_fp_fn(ground_truths, detections_after_classifier, iou_threshold))

TP 297
recall 0.871
precision 0.616
f_score 0.722


## Resize + KNN

In [35]:
detections_after_classifier = []
predictions = []

for detection in tqdm(detections):

    img = cv2.imread(f_names[index2fname[detection[0]]]['path'])
    x_min, y_min, x_max, y_max = [int(i) for i in detection[3:]]

    crop_img = img[y_min:y_max, x_min:x_max]
    
    prediction = classify_crop(crop_img=crop_img, model=resnet50_resize_knn, knn=knn,
                    transform=data_transform_resize)
    predictions.append(prediction)
    
    # Вызываем классификатор
    if classify_crop(crop_img=crop_img, model=resnet50_resize_knn, knn=knn,
                    transform=data_transform_resize) > 0.5:
        detections_after_classifier.append(detection)

100%|█████████████████████████████████████████| 482/482 [04:33<00:00,  1.76it/s]


In [36]:
print_main_metrics(*calc_tp_fp_fn(ground_truths, detections_after_classifier, iou_threshold))

TP 297
recall 0.871
precision 0.624
f_score 0.727
